# Deep Convolutional GAN

### Import Tensorflow & Download data

In [1]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", reshape=False, validation_size=0)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Build Network

#### Inputs

In [3]:
def model_inputs(real_dim, z_dim):
    input_real = tf.placeholder(tf.float32, (None, *real_dim), name='input_real')
    input_z = tf.placeholder(tf.float32, (None, z_dim), name='input_z')
    return input_real, input_z

#### Generator

In [4]:
def generator(z, reuse=False, alpha=0.2, training=True):
    with tf.variable_scope('generator', reuse=reuse):
        # Layer 0
        layer_0 = tf.layers.dense(z, 2*2*128)
        layer_0 = tf.reshape(layer_0, (-1, 2, 2, 128))
        layer_0 = tf.layers.batch_normalization(layer_0, training=training)
        layer_0 = tf.maximum(alpha * layer_0, layer_0)
        # Size 2x2x128
        
        # Layer 1
        layer_1 = tf.layers.conv2d_transpose(layer_0, 64, 5, strides=2, padding='valid')
        layer_1 = tf.layers.batch_normalization(layer_1, training=training)
        layer_1 = tf.maximum(alpha * layer_1, layer_1)
        # Size 7x7x64
        
        # Layer 2
        layer_2 = tf.layers.conv2d_transpose(layer_1, 32, 5, strides=2, padding='same')
        layer_2 = tf.layers.batch_normalization(layer_2, training=training)
        layer_2 = tf.maximum(alpha * layer_2, layer_2)
        # Size 14x14x32
        
        # Output
        logits = tf.layers.conv2d_transpose(layer_2, 1, 5, strides=2, padding='same')
        out = tf.tanh(logits)
        # Size 28x28x1
        
        return out

#### Discriminator

In [5]:
def discriminator(x, reuse=False, alpha=0.2, training=True):
    with tf.variable_scope('discriminator', reuse=reuse):
        # x: 28x28x1
        
        # Layer 0 is 
        layer_0 = tf.layers.conv2d(x, 32, 5, strides=2, padding='same')
        layer_0 = tf.maximum(alpha * layer_0, layer_0)
        # Size 14x14x32
        
        # Layer 1
        layer_1 = tf.layers.conv2d(layer_0, 64, 5, strides=2, padding='same')
        layer_1 = tf.layers.batch_normalization(layer_1, training=training)
        layer_1 = tf.maximum(alpha * layer_1, layer_1)
        # Size 7x7x64
        
        # layer_2 = tf.layers.conv2d(layer_1, 128, 5, strides=2, padding='same')
        # layer_2 = tf.layers.batch_normalization(layer_2, training=training)
        # layer_2 = tf.maximum(alpha * layer_2, layer_2)
        # Size 4x4x128

        # Flatten it
        flat = tf.reshape(layer_1, (-1, 7*7*64))
        logits = tf.layers.dense(flat, 1)
        out = tf.sigmoid(logits)
        
        return out, logits

#### Hyper Parameters

In [6]:
input_size = (28, 28, 1)
z_size = 100
learning_rate = 0.01
batch_size = 128
epochs = 15
alpha = 0.2
# Smoothing
smooth = 0.05

#### Define Model

In [7]:
tf.reset_default_graph()

input_real, input_z = model_inputs(input_size, z_size)

# Generator
with tf.name_scope('generator'):
    g_model = generator(input_z, alpha=alpha)

# Discriminator
with tf.name_scope('discriminator_real'):
    d_model_real, d_logits_real = discriminator(input_real, alpha=alpha)
with tf.name_scope('discriminator_fake'):
    d_model_fake, d_logits_fake = discriminator(g_model, alpha=alpha, reuse=True)

#### Loss Functions

In [8]:
with tf.name_scope('g_loss'):
    g_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.ones_like(d_logits_fake)))

with tf.name_scope('d_loss'):
    with tf.name_scope('d_loss_real'):
        d_loss_real = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, labels=tf.ones_like(d_logits_real) * (1 - smooth)))
    with tf.name_scope('d_loss_fake'):
        d_loss_fake = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.zeros_like(d_logits_fake)))

    d_loss = d_loss_real + d_loss_fake

#### Optimizers

In [9]:
t_vars = tf.trainable_variables()
g_vars = [var for var in t_vars if var.name.startswith('generator')]
d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    g_train_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)
    d_train_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)

## Training

In [10]:
samples = []
losses = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for e in range(epochs):
        for ii in range(mnist.train.num_examples//batch_size):
            # Batch Images
            batch_images = mnist.train.next_batch(batch_size)[0]
            # Random Noise
            batch_z = np.random.uniform(-1, 1, size=(batch_size, z_size))
            
            # Optimize
            _ = sess.run(g_train_opt, feed_dict={input_real: batch_images, input_z: batch_z})
            _ = sess.run(d_train_opt, feed_dict={input_real: batch_images, input_z: batch_z})
        
        g_train_loss = sess.run(g_loss, feed_dict={input_z: batch_z})
        d_train_loss = sess.run(d_loss, feed_dict={input_real: batch_images, input_z: batch_z})
        
        print(
            "Epoch {:2d}/{}...".format(e+1, epochs),
            "Generator Loss: {:.3f}...".format(g_train_loss),
            "Discriminator Loss: {:.3f}".format(d_train_loss),
        )
        
        losses.append((g_train_loss, d_train_loss))
        
        sample_z = np.random.uniform(-1, 1, size=(16, z_size))
        gen_samples = sess.run(g_model, feed_dict={input_z: sample_z})
        
        samples.append(gen_samples)

Epoch  1/15... Generator Loss: 5.876... Discriminator Loss: 0.396
Epoch  2/15... Generator Loss: 4.488... Discriminator Loss: 0.275
Epoch  3/15... Generator Loss: 5.776... Discriminator Loss: 0.254
Epoch  4/15... Generator Loss: 5.766... Discriminator Loss: 0.236
Epoch  5/15... Generator Loss: 6.671... Discriminator Loss: 0.213
Epoch  6/15... Generator Loss: 5.251... Discriminator Loss: 0.245
Epoch  7/15... Generator Loss: 5.399... Discriminator Loss: 0.233
Epoch  8/15... Generator Loss: 6.234... Discriminator Loss: 0.223
Epoch  9/15... Generator Loss: 6.009... Discriminator Loss: 0.212
Epoch 10/15... Generator Loss: 5.911... Discriminator Loss: 0.216
Epoch 11/15... Generator Loss: 4.897... Discriminator Loss: 0.218
Epoch 12/15... Generator Loss: 4.349... Discriminator Loss: 0.226
Epoch 13/15... Generator Loss: 6.389... Discriminator Loss: 0.218
Epoch 14/15... Generator Loss: 6.375... Discriminator Loss: 0.215
Epoch 15/15... Generator Loss: 4.837... Discriminator Loss: 0.223


In [ ]:
fig, ax = plt.subplots()
losses = np.array(losses)
plt.plot(losses.T[0], label='Generator')
plt.plot(losses.T[1], label='Distriminator')
plt.title("Training Losses")
plt.legend()

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    file_writer = tf.summary.FileWriter('./logs/1', sess.graph)